In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn 
from datetime import datetime

### Demo of BTC Difficulty Adjustment
####    &nbsp;  OE Skunk Works    &nbsp;  &nbsp;       27nov20        -dbb

In [2]:
import psycopg2 as pg
import pandas.io.sql as psql

conn  = pg.connect("dbname=btc_hist")

    btc_hist=# \d data_chain
           Table "public.data_chain"
        Column     |       Type       |  
    ---------------+------------------+
     height_str    | text             |
     blkhash       | text             |
     bits_hex      | text             |
     difficulty    | double precision |
     chainwork_hex | text             |
     time_str      | text             |
     totaltime_str | text             |

--

     class CBlockHeader {  public:
       // header
       int32_t   nVersion;
       uint256   hashPrevBlock;
       uint256   hashMerkleRoot;
       uint32_t  nTime;
       uint32_t  nBits;
       uint32_t  nNonce;
       

In [3]:
tSQL = ''' SELECT
      height_str::bigint as blk_ht,
      bits_hex,
      difficulty,
      time_str::bigint as blk_time_longint,
      totaltime_str::bigint as blk_totaltime_longint
FROM
    data_chain   ORDER BY   height_str::bigint
'''
df_datachain = pd.read_sql_query( tSQL,con=conn )

In [4]:
df_datachain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649600 entries, 0 to 649599
Data columns (total 5 columns):
blk_ht                   649600 non-null int64
bits_hex                 649600 non-null object
difficulty               649600 non-null float64
blk_time_longint         649600 non-null int64
blk_totaltime_longint    649600 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB


In [5]:
## Note:  by examination, the first adjusted BTC difficulty was 
##  the 15th 2-week interval .. so demo-calc that interval
##  the start offset is one less than a full cycle (2015)

tbase = 15
##  constant for 2-weeks of mining, defined for the network at init time
dif_interval = 6*24*14    # =>2016 ten minute blocks     

## convenience variables to mark start and end in the interval, a.k.a. Regimen
sbeg = (dif_interval * tbase) + 2015   ## skip the genesis interval in all calcs
send = sbeg + dif_interval             ## the last block with the same difficulty

##  df_datachain[ sbeg:send ]

##  a full 2016 row interval is too long to show inline, so..
##  instead, show the first four rows, 
##    then the last four ending exactly on the last blk in the regimine
df_datachain[ sbeg:(sbeg+4) ]

,blk_ht,bits_hex,difficulty,blk_time_longint,blk_totaltime_longint
32255,32256,0x1d00d86a,1.1829,1262153464,1262150214
32256,32257,0x1d00d86a,1.1829,1262154352,1262150242
32257,32258,0x1d00d86a,1.1829,1262155561,1262151952
32258,32259,0x1d00d86a,1.1829,1262156580,1262152015


In [6]:
df_datachain[ (send-4):(send) ]

,blk_ht,bits_hex,difficulty,blk_time_longint,blk_totaltime_longint
34267,34268,0x1d00d86a,1.1829,1263248530,1263246631
34268,34269,0x1d00d86a,1.1829,1263248863,1263247097
34269,34270,0x1d00d86a,1.1829,1263249045,1263247477
34270,34271,0x1d00d86a,1.1829,1263249842,1263247807


###  Compute BITS Difficulty Change   &nbsp; &nbsp; method-0

In [42]:
##
##  Method Zero recreates the C++ source code found in  pow.cpp
##   The main change from method-1 is eliminate the float multiply.
##
##  08dec20  match; seven discrepancies solved   -dbb
##


def unpack_exp(x):
    ## defined in the compact BITS format
    return 8*(x - 3)

def getCompact( inLongInt ):
    ##  inLongInt -> long integer  uint256
    ##  returns a 4 byte BITS compact representation
    #assert( str(type(inLongInt)) == "<type 'long'>" )
    
    ## get bytes needed to represent this number
    cnt = ( (inLongInt.bit_length()+7)/8  )
    
    ## get an integer from inputs most-significant three bytes 
    resInt = inLongInt >> ( (cnt*8) - (3*8) )
    #print '  resInt:  ' +hex(resInt)
    
    ## build a mask to test the high bit of the large integer input
    ##  python-specific, this must result in a whole number of bytes
    shft_cnt = inLongInt.bit_length()
    if (shft_cnt % 8)!=0:  shft_cnt += (8-(shft_cnt % 8))

    ## mask test the high bit of the input
    ##  the top bit of the third byte in compact format is RESERVED
    ##  check if the result has this bit set, if so
    ##    shift the value content right (losing precision) 
    ##    increment the total number of bytes of padding
    if (0x80 << (shft_cnt-8)) & inLongInt :
        #print 'big8'
        resInt = resInt >> (1*8)
        cnt += 1

    
    ## result -> compact BITS format  (4 bytes)
    ##  top byte -> count of bytes in the expanded format
    ##  lower three bytes -> integer value
    resInt = (cnt << (3*8))  | resInt
    return resInt


In [44]:
##
##  for a given Regimen ID# 
##
##        bits_hex: the BITS field in the BTC blockchain
##     exp_const:  expanded uint256 value from that BITS field
##
##    calc_diff_full:  predicted TARGET value, defines a new difficulty
##   calc_diff_compact:  predicted BITS compact for calc_diff_full
##
##  e.g.  Reg# 220  predicted BITS_hex: 0x18034379
##        Reg# 221  shows actual  BITS_hex: 0x18034379
##
##---------------------------------------------------------------------------------


tbase = 0
## well-known constant for 2-weeks of mining
dif_interval = 6*24*14    # =>2016 ten minute blocks     

## convenience variables to mark start and end in the interval
sbeg = (dif_interval * tbase) + 2015   ## skip the genesis interval in all calcs
send = sbeg + dif_interval             ## the last block with the same difficulty

elem  = sbeg
cnt   = 0

while cnt < len(df_datachain):
    ## note: time calc considers only blocks leading to the last block
    try:
        blk_start_ts = df_datachain['blk_time_longint'][sbeg]
        blk_end_ts =  df_datachain['blk_time_longint'][send-1]
    except:
        break
    blk_interval_secs =  blk_end_ts - blk_start_ts

    ##--
    print 'Reg# '+str(tbase+cnt)+'     blk_ht: '+ str(df_datachain['blk_ht'][sbeg]) 
    print '        seconds: '+str(blk_interval_secs)
    print " from "+datetime.utcfromtimestamp(float(blk_start_ts)).strftime("%Y-%m-%d %H:%M:%S")
    print "   to "+datetime.utcfromtimestamp(float(blk_end_ts)).strftime("%Y-%m-%d %H:%M:%S")
    print " "

    ## retrieve compact BITS string for this Regimen; extract parts
    bits_hex = int( df_datachain['bits_hex'][sbeg], 16) 
    reg_difficulty  = bits_hex & 0x007FFFFF
    reg_exp = (bits_hex & 0xFF000000) >> 24
    print '      bits_hex: '+hex(bits_hex)+'  | '+hex(reg_exp)+'  '+hex(reg_difficulty)

    ## calc the ratio of (actual seconds) / (ideal seconds) for this regimen
    ##  ratio < 1.0 shows the engine is producing too many blocks
    ##   therefore increase the difficulty to make new blocks
    ##  ratio > 1.0 shows the engine did not produce enough blocks
    ##   therefore decrease the difficulty to make a new block
    trt = blk_interval_secs / (2016*600.0)

    if int(df_datachain['blk_ht'][sbeg]) < 30000:  tind = 'NO_CHANGE'
    elif trt > 1.0:  tind = 'DECREASE'
    else:  tind = 'INCREASE'  

    #  extract target value     TODO use factors
    exp_const = reg_difficulty * 2**unpack_exp(reg_exp)
    exp_bits =  1 * 2**unpack_exp(reg_exp)
    #print "exp_const bit length: "+ str(exp_const.bit_length())
    #print "exp_const byte length: "+ str((exp_const.bit_length()+7)/8)
    #print "exp_bits byte length: "+ str((exp_bits.bit_length()+7)/8)
    print "     exp_const: " + hex(exp_const)[:-1] + "\n"
    
    # calc new hex value in two steps, using integers 
    MBFLAG = True
    if MBFLAG:
      calc_diff = blk_interval_secs * exp_const
      calc_diff_pre = calc_diff / (2016*600)
      print "calc_diff_full: " + hex(calc_diff_pre)[:-1] 
      calc_diff_post = getCompact( calc_diff_pre )
      print "calc_diff_compact: " + hex(calc_diff_post)[:-1] + "\n"
      print "              Difficulty -> " + tind 

      #calc_diff_long = calc_diff / (2016*600)
      #print "calc_diff_long  bytes: "+ str((calc_diff_long.bit_length()+7)/8)
      #print "calc_diff byte length: "+ str((calc_diff.bit_length()+7)/8)
    else:
      calc_diff = blk_interval_secs * reg_difficulty
      calc_diff = calc_diff / (2016*600)
      print " Difficulty -> " + tind + " calc_diff: " + hex(calc_diff)[:-1] + "\n"
    
    print "------------------------------------"
    
    cnt = cnt + 1
    sbeg = sbeg + 2016
    send = send + 2016
    #if cnt == 36: break


Reg# 0     blk_ht: 2016
        seconds: 1401591
 from 2009-01-27 13:38:51
   to 2009-02-12 18:58:42
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
     exp_const: 0xffff0000000000000000000000000000000000000000000000000000

calc_diff_full: 0x128a0e4b1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1fb1
calc_diff_compact: 0x1d0128a0

              Difficulty -> NO_CHANGE
------------------------------------
Reg# 1     blk_ht: 4032
        seconds: 1499277
 from 2009-02-12 19:16:30
   to 2009-03-02 03:44:27
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
     exp_const: 0xffff0000000000000000000000000000000000000000000000000000

calc_diff_full: 0x13d4d6e02702702702702702702702702702702702702702702702702
calc_diff_compact: 0x1d013d4d

              Difficulty -> NO_CHANGE
------------------------------------
Reg# 2     blk_ht: 6048
        seconds: 1540887
 from 2009-03-02 04:01:53
   to 2009-03-20 00:03:20
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
     exp_const: 0xffff000000000000000000000000

 
      bits_hex: 0x18171a8b  | 0x18  0x171a8b
     exp_const: 0x171a8b000000000000000000000000000000000000000000

calc_diff_full: 0x162043e9f4e4a39f8f4e4a39f8f4e4a39f8f4e4a39f8f4e4
calc_diff_compact: 0x18162043

              Difficulty -> INCREASE
------------------------------------
Reg# 178     blk_ht: 360864
        seconds: 1216710
 from 2015-06-14 04:56:03
   to 2015-06-28 06:54:33
 
      bits_hex: 0x18162043  | 0x18  0x162043
     exp_const: 0x162043000000000000000000000000000000000000000000

calc_diff_full: 0x16418e5d8138138138138138138138138138138138138138
calc_diff_compact: 0x1816418e

              Difficulty -> DECREASE
------------------------------------
Reg# 179     blk_ht: 362880
        seconds: 1169948
 from 2015-06-28 07:07:26
   to 2015-07-11 20:06:34
 
      bits_hex: 0x1816418e  | 0x18  0x16418e
     exp_const: 0x16418e000000000000000000000000000000000000000000

calc_diff_full: 0x1586c85cbb076c327ee3a9f65b216dd298e54a105cc187d4
calc_diff_compact: 0x181586c8

   

              Difficulty -> INCREASE
------------------------------------
Reg# 289     blk_ht: 584640
        seconds: 1216360
 from 2019-07-09 09:17:48
   to 2019-07-23 11:10:28
 
      bits_hex: 0x171f0d9b  | 0x17  0x1f0d9b
     exp_const: 0x1f0d9b0000000000000000000000000000000000000000

calc_diff_full: 0x1f3a08675fa0a4b4f5fa0a4b4f5fa0a4b4f5fa0a4b4f5f
calc_diff_compact: 0x171f3a08

              Difficulty -> DECREASE
------------------------------------
Reg# 290     blk_ht: 586656
        seconds: 1091908
 from 2019-07-23 11:22:17
   to 2019-08-05 02:40:45
 
      bits_hex: 0x171f3a08  | 0x17  0x1f3a08
     exp_const: 0x1f3a080000000000000000000000000000000000000000

calc_diff_full: 0x1c3039cb208fe6dc4ba298075e53c31a0f7ed5cb3a9186
calc_diff_compact: 0x171c3039

              Difficulty -> INCREASE
------------------------------------
Reg# 291     blk_ht: 588672
        seconds: 1186065
 from 2019-08-05 02:52:08
   to 2019-08-18 20:19:53
 
      bits_hex: 0x171c3039  | 0x17  0x1c303

In [9]:
0xba18 * 0.68813  # 32782.5132
hex(32782)  #  0x800e
#--
0x10c5a  * 0.69  # 47401.619999999995
hex(47401)  #  0xb929

#--
#0x7FFF   # 32767


'0xb929'

In [9]:
%whos

Variable            Type          Data/Info
-------------------------------------------
MBFLAG              bool          True
bits_hex            int           386939410
blk_end_ts          int64         1600569231
blk_interval_secs   int64         1086311
blk_start_ts        int64         1600570533
calc_diff           long          1688368698599932972367448<...>8428006979540066251046912
calc_diff_post      int           0
calc_diff_pre       long          1395807455853119190118591<...>0375717946434341550980696
cnt                 int           321
conn                connection    <connection object at 0x7<...>ame=btc_hist', closed: 0>
datetime            type          <type 'datetime.datetime'>
df_datachain        DataFrame             blk_ht    bits_he<...>[649600 rows x 5 columns]
dif_interval        int           2016
elem                int           2015
exp_bits            long          1461501637330902918203684832716283019655932542976
exp_const           long          155422

In [47]:
send = len(df_datachain)-1
sbeg = send - 2016

blk_start_ts = df_datachain['blk_time_longint'][sbeg]
blk_end_ts =  df_datachain['blk_time_longint'][send]
blk_interval_secs =  blk_end_ts - blk_start_ts

print "time taken to make 2016 blocks : "
print " from "+datetime.utcfromtimestamp(float(blk_start_ts)).strftime("%Y-%m-%d %H:%M:%S")
print "   to "+datetime.utcfromtimestamp(float(blk_end_ts)).strftime("%Y-%m-%d %H:%M:%S")
print " "
print "       " + str(blk_interval_secs)+ ' seconds'
print " " + str(blk_interval_secs/60.0 )+' minutes'
print " " + str((blk_interval_secs/60.0)/60.0 )+' hours'
print " " + str(((blk_interval_secs/60.0)/60.0)/24)+' days'
print " "
print " " + str(blk_interval_secs/(14*24*6.0))+ ' seconds per block'
print " "

if blk_interval_secs > (14*24*6*600):  tind = 'DECREASE'
else:  tind = 'INCREASE'
    
print " Difficulty -> " + tind

time taken to make 2016 blocks : 
 from 2020-09-10 09:39:04
   to 2020-09-23 09:31:16
 
       1122732 seconds
 18712.2 minutes
 311.87 hours
 12.9945833333 days
 
 556.910714286 seconds per block
 
 Difficulty -> INCREASE


### Method 2:  Take an average without SUM()

new average = ((old_count * old_data) + next_data) / next_count

new average = old_average + (next_data - old_average) / next_count

In [8]:
## per Difficulty Formula, compare an ideal time to actual time
##   https://en.bitcoin.it/wiki/Difficulty#
##     How_is_difficulty_calculated.3F_What_is_the_difference_between_bdiff_and_pdiff.3F

In [9]:
## calc and emit the mean interval for series blk_time_longint
##    data_chain [ sbeg  to  send  ] 
##

elem  = sbeg
cnt   = 1
## first elem assign an ideal time 60*10
t_avg = 600.0  

while True:
  if cnt == 1:
    elem = elem+1
    cnt  = cnt+1
    continue

  d_elem = df_datachain['blk_time_longint'][elem]
  dp_elem = df_datachain['blk_time_longint'][elem-1]
  time_diff = d_elem - dp_elem
    
  dx = time_diff - t_avg
  new_avg = (t_avg + ( time_diff - t_avg)/ float(cnt))

  if cnt == send - sbeg:
    print ' avg blk time: '+str(new_avg)
    break

  #print ' '+str(elem)+'    '+str(t_avg)+'           '+str(dx)+'     '+str(new_avg)
  t_avg = new_avg
  elem = elem + 1
  cnt = cnt + 1
  # end While


 avg blk time: 544.135912698


### Calc Mean Mining Time for the Chain by Difficulty Regimen

In [10]:
## calc and emit the mean interval for series blk_time_longint
##    data_chain [ sbeg  to  send  ] 
##

tbase = 15
## well-known constant for 2-weeks of mining
dif_interval = 6*24*14    # =>2016 ten minute blocks     

## convenience variables to mark start and end in the interval
sbeg = (dif_interval * tbase) + 2015   ## skip the genesis interval in all calcs
send = sbeg + dif_interval             ## the last block with the same difficulty

elem  = sbeg
cnt   = 0

## Simulate the first avg as an ideal time 60*10
t_avg = 600.0  
elem = elem + 1
cnt  = cnt + 1

print "blk_ht,mean_blk_time"
while True:
  d_elem = df_datachain['blk_time_longint'][elem]
  dp_elem = df_datachain['blk_time_longint'][elem-1]
  time_diff = d_elem - dp_elem
    
  dx = time_diff - t_avg
  new_avg = (t_avg + ( time_diff - t_avg)/ float(cnt))

  #print ' '+str(elem)+'    '+str(t_avg)+'           '+str(dx)+'     '+str(new_avg)
  t_avg = new_avg

  if cnt == dif_interval:
    ## pesky list index to blk_ht +1
    print  str(elem-dif_interval+1)+','+str(new_avg)
    cnt = 0
    t_avg = 600.0
    
  elif elem == (len(df_datachain)-1):
    break

  elem = elem + 1
  cnt = cnt + 1

# end While


blk_ht,mean_blk_time
32256,543.974702381
34272,582.719246032
36288,443.90625
38304,432.433035714
40320,401.431051587
42336,500.996527778
44352,595.828869048
46368,450.385912698
48384,467.035714286
50400,409.318452381
52416,535.517857143
54432,650.822916667
56448,428.081845238
58464,574.076388889
60480,537.529761905
62496,495.544642857
64512,310.899305556
66528,143.62202381
68544,446.814980159
70560,416.188988095
72576,413.232142857
74592,492.922619048
76608,524.745039683
78624,466.050595238
80640,417.688988095
82656,574.30406746
84672,385.064484127
86688,417.426587302
88704,409.841765873
90720,396.87797619
92736,510.516369048
94752,395.896329365
96768,507.560019841
98784,533.218253968
100800,530.831349206
102816,503.275793651
104832,508.248511905
106848,429.242063492
108864,394.150793651
110880,438.291170635
112896,662.911706349
114912,502.68452381
116928,535.108630952
118944,505.458333333
120960,418.583333333
122976,387.124503968
124992,336.819940476
127008,460.212797619
129024,388.22

    btc_hist=# select d.height_str, d.bits_hex,c.mean_blk_time from data_chain d JOIN     chng_blk_time c ON (d.height_str::integer = c.blk_height::integer) order by     c.blk_height::integer;
    
    btc_hist=# create table blk_times as select d.height_str::integer as blk_height, d.bits_hex,c.mean_blk_time from data_chain d JOIN chng_blk_time c ON (d.height_str::integer = c.blk_height::integer) order by c.blk_height::integer;
    SELECT 306
    
    btc_hist=# \d blk_times
                     Table "public.blk_times"
        Column     |       Type       | Collation | Nullable | Default 
    ---------------+------------------+-----------+----------+---------
     blk_height    | integer          |           |          | 
     bits_hex      | text             |           |          | 
     mean_blk_time | double precision |           |          | 



In [11]:
bits_hex = int( df_datachain['bits_hex'][sbeg], 16) 
reg_difficulty  = bits_hex & 0x00FFFFFF
reg_exp = (bits_hex & 0xFF000000) >> 24
print str(type(reg_difficulty))+'  '+hex(reg_exp)+' '+hex(reg_difficulty)

<type 'int'>  0x1d 0xd86a


###  Compute BITS Difficulty Change   &nbsp; &nbsp; method-1

In [68]:
##
##  ratio 'trt' of:  actual_time / ideal time
##
##    ratio > 1.0   => difficulty DECREASE; bits_hex INCREASE 
##
##    ratio < 1.0   => difficulty INCREASE; bits_hex DECREASE 
##
##   At least one problem with this solution, this simple
##   code ignores the exponent portion of the BITS format, which will 
##   occassionally lead to a very wrong result, but most often has no effect.
## e.g.  Reg# 36,59,122,147,248
##   (since there are actually only 300-odd times the value changes, a manual
##   conditional test could be used instead of tricky factors methods).
##
##  Read the answer for a given Regimen ID# by  last hex value on the right,
##   compared to first hex value on the right, in the next Regimen below it.
##
##  e.g.  Reg# 220  has predicted change  trt_ratio*BITS: 0x343e4
##        Reg# 221  below has actual  BITS_hex  0x34379
##

tbase = 0
## well-known constant for 2-weeks of mining
dif_interval = 6*24*14    # =>2016 ten minute blocks     

## convenience variables to mark start and end in the interval
sbeg = (dif_interval * tbase) + 2015   ## skip the genesis interval in all calcs
send = sbeg + dif_interval             ## the last block with the same difficulty

elem  = sbeg
cnt   = 0

while cnt < len(df_datachain):
    print 'Reg# '+str(tbase+cnt)+'     blk_ht: '+ str(df_datachain['blk_ht'][sbeg]) 
    print '           seconds: '+str(blk_interval_secs)
    print " from "+datetime.utcfromtimestamp(float(blk_start_ts)).strftime("%Y-%m-%d %H:%M:%S")
    print "   to "+datetime.utcfromtimestamp(float(blk_end_ts)).strftime("%Y-%m-%d %H:%M:%S")
    print " "

    bits_hex = int( df_datachain['bits_hex'][sbeg], 16) 
    reg_difficulty  = bits_hex & 0x00FFFFFF
    reg_exp = (bits_hex & 0xFF000000) >> 24
    print '      bits_hex: '+hex(bits_hex)+'  | '+hex(reg_exp)+'  '+hex(reg_difficulty)

    try:
        blk_start_ts = df_datachain['blk_time_longint'][sbeg]
        blk_end_ts =  df_datachain['blk_time_longint'][send]
    except:
        break
    blk_interval_secs =  blk_end_ts - blk_start_ts

    trt = blk_interval_secs / (2016*600.0)
    
    print 'reg_difficulty:  '+str(reg_difficulty)
    print '     trt_ratio: '+str(trt) +'  trt_ratio*BITS: '+hex( trt*reg_difficulty )[:-1] + '\n'
    # calc new hex value, or

    if trt > 1.0:  tind = 'DECREASE'
    else:  tind = 'INCREASE'  
    print " Difficulty -> " + tind + "\n"
    
    print "-----------------------------"
    
    cnt = cnt + 1
    sbeg = sbeg + 2016
    send = send + 2016


Reg# 0     blk_ht: 2016
           seconds: 1088957
 from 2020-09-20 02:55:33
   to 2020-09-20 03:17:57
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
reg_difficulty:  65535
     trt_ratio: 1.15960565476  trt_ratio*BITS: 0x128da

 Difficulty -> DECREASE

-----------------------------
Reg# 1     blk_ht: 4032
           seconds: 1402659
 from 2009-01-27 13:38:51
   to 2009-02-12 19:16:30
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
reg_difficulty:  65535
     trt_ratio: 1.2403463955  trt_ratio*BITS: 0x13d86

 Difficulty -> DECREASE

-----------------------------
Reg# 2     blk_ht: 6048
           seconds: 1500323
 from 2009-02-12 19:16:30
   to 2009-03-02 04:01:53
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
reg_difficulty:  65535
     trt_ratio: 1.27502728175  trt_ratio*BITS: 0x14666

 Difficulty -> DECREASE

-----------------------------
Reg# 3     blk_ht: 8064
           seconds: 1542273
 from 2009-03-02 04:01:53
   to 2009-03-20 00:26:26
 
      bits_hex: 0x1d00ffff  | 0x1d  0xffff
reg

   to 2015-05-31 19:06:25
 
      bits_hex: 0x18171a8b  | 0x18  0x171a8b
reg_difficulty:  1514123
     trt_ratio: 0.957819113757  trt_ratio*BITS: 0x16210f

 Difficulty -> INCREASE

-----------------------------
Reg# 178     blk_ht: 360864
           seconds: 1158578
 from 2015-05-31 19:06:25
   to 2015-06-14 04:56:03
 
      bits_hex: 0x18162043  | 0x18  0x162043
reg_difficulty:  1450051
     trt_ratio: 1.00651703042  trt_ratio*BITS: 0x16452d

 Difficulty -> DECREASE

-----------------------------
Reg# 179     blk_ht: 362880
           seconds: 1217483
 from 2015-06-14 04:56:03
   to 2015-06-28 07:07:26
 
      bits_hex: 0x1816418e  | 0x18  0x16418e
reg_difficulty:  1458574
     trt_ratio: 0.968121693122  trt_ratio*BITS: 0x158bed

 Difficulty -> INCREASE

-----------------------------
Reg# 180     blk_ht: 364896
           seconds: 1171040
 from 2015-06-28 07:07:26
   to 2015-07-11 20:24:46
 
      bits_hex: 0x181586c8  | 0x18  0x1586c8
reg_difficulty:  1410760
     trt_ratio: 0.977181

In [39]:
##-- SCRATCH vv -------------------------------------------------

In [16]:
## EXTRACT only blks when difficulty changes

cnt = 1
cur_bits = df_datachain['bits_hex'][cnt]

seq_bitsL  = [ ]
seq_bitsL.append(cur_bits)
seq_indexL = [ ]
seq_indexL.append(1)
ind_seq = 0

while True:
    if seq_bitsL[-1] != df_datachain['bits_hex'][cnt] :
        seq_bitsL.append( df_datachain['bits_hex'][cnt])
        # pesky convert from list index to blk_ht +1
        seq_indexL.append( cnt+1 )
    
    cnt = cnt + 1
    if cnt == len(df_datachain):
        break

print " seq_bitsL len: " + str(len(seq_bitsL))
print " seq_indexL len: " + str(len(seq_indexL))

 seq_bitsL len: 308
 seq_indexL len: 308


In [5]:
544.0 / 600.0
0xd86a * 0.906

50194.212

In [6]:
hex(50194)

'0xc412'

In [17]:
cnt = 0
while True:
    print str(seq_indexL[cnt])+','+str(seq_bitsL[cnt])
    cnt = cnt+1
    if cnt == len(seq_indexL):  break

1,0x1d00ffff
32256,0x1d00d86a
34272,0x1d00c428
36288,0x1d00be71
38304,0x1d008cc3
40320,0x1c654657
42336,0x1c43b3e5
44352,0x1c387f6f
46368,0x1c381375
48384,0x1c2a1115
50400,0x1c20bca7
52416,0x1c16546f
54432,0x1c13ec53
56448,0x1c159c24
58464,0x1c0f675c
60480,0x1c0eba64
62496,0x1c0d3142
64512,0x1c0ae493
66528,0x1c05a3f4
68544,0x1c0168fd
70560,0x1c010c5a
72576,0x1c00ba18
74592,0x1c00800e
76608,0x1b692098
78624,0x1b5bede6
80640,0x1b4766ed
82656,0x1b31b2a3
84672,0x1b2f8e9d
86688,0x1b1e7eca
88704,0x1b153263
90720,0x1b0e7256
92736,0x1b098b2a
94752,0x1b081cd2
96768,0x1b055953
98784,0x1b04864c
100800,0x1b0404cb
102816,0x1b038dee
104832,0x1b02fa29
106848,0x1b028552
108864,0x1b01cc26
110880,0x1b012dcd
112896,0x1b00dc31
114912,0x1b00f339
116928,0x1b00cbbd
118944,0x1b00b5ac
120960,0x1b0098fa
122976,0x1a6a93b3
124992,0x1a44b9f2
127008,0x1a269421
129024,0x1a1d932f
131040,0x1a132185
133056,0x1a0c2a12
135072,0x1a0abbcf
137088,0x1a09ec04
139104,0x1a08e1e5
141120,0x1a094a86
143136,0x1a096fe3
145152,0x1a09

In [ ]:
54431 - 52415


print 34271 - 32255
print 36287 - 34271
print 38303 - 36287
print 40319 - 38303
print 42335 - 40319
print 44351 - 42335
print 46367 - 44351
print 48383 - 46367 
print 50399 - 48383
print 52415 - 50399

In [ ]:
(32255-2015) / 2016.0


In [ ]:
#seq_bitsL[0:12]
#seq_bitsL
seq_indexL
#len(df_datachain)

In [ ]:
tbase = 16

## convenience variables to mark start and end in the interval
sbeg = (dif_interval * tbase) + 1   ## skip the genesis block in all calcs
send = sbeg + dif_interval 

##  df_datachain[ sbeg:send ]

##  a full interval is too long to show at once inline, so..
##  instead, show the first four rows, 
##    then the last four plus the changed next one
df_datachain[ sbeg:(sbeg+4) ]

In [ ]:
df_datachain[ (send-4):(send+4) ]

In [ ]:
##  Show the accumulated average code with a simple example

test_d = [ 5, 5, 6, 6, 7, 8, 2]        ## avg = 39/7 = 5.571428571428571
# avgs in order:   5    5   5.33   5.5   5.8   6.16    5.571428571428571

elem = 0
cnt = 1

print 'elem'+' '+'t_avg'+'         '+'dx'+'     '+'new_avg'

for val in test_d:
  if elem == 0:
    t_avg = test_d[0]
    elem = elem+1
    cnt = cnt+1
    continue

  if elem == 1:
    t_avg = (test_d[0]+test_d[1]) / 2.0
    elem = elem+1
    cnt = cnt+1
    continue

  if cnt == len(test_d):
    new_d = test_d[elem]
    new_avg = (t_avg + ( new_d -t_avg )/ float(cnt) )
    print 'Final Avg: '+str(new_avg)
    break

  new_d = test_d[elem]
  dx = new_d - t_avg
  new_avg = (t_avg + ( new_d -t_avg )/ float(cnt) )
  print ' '+str(elem)+'    '+str(t_avg)+'           '+str(dx)+'     '+str(new_avg)
  t_avg = new_avg
  elem = elem + 1
  cnt = cnt+1


In [ ]:
#s1 = Series( df_datachain[''], index = df_datachain[''] )

s1 = pd.Series( df_datachain['difficulty'], index = df_datachain['blk_ht'] )

#s1[1:2016].plot()


In [ ]:
tbase = 15
dif_interval = 2016

sbeg = 2016 * tbase
send = sbeg + 2016 

##  plot one two-week interval; does not show the change in difficulty
## s1[ sbeg:send ].plot()

##  plot one two-week interval, plus four on either side to show change direction
s1[ (sbeg-4):(4+send) ].plot()

--=======================================================

In [ ]:
## take a float value and make a histogram
##  hist() is not a sequence, it is a summary, grouped by value
df_datachain['difficulty'].hist()

In [ ]:
##  Not Great - height_str gets sorted in alpha order
# df_datachain.set_index('height_str')

In [ ]:
## extract all rows where a value is less than MAX
df_smd = df_datachain[   df_datachain['difficulty'] < 80000   ]
df_smd.info()

In [ ]:
df_smd['difficulty' ].plot()

In [ ]:
## start over .. do the conversion in SQL
tSQL = 'select \
  height_str::integer as blk_height, \
  difficulty, \
  chainwork_hex, time_str::bigint as time_b, \
  get_fmt_date(time_str) as date_str, \
  totaltime_str::bigint as ttime_b \
  from data_chain order by height_str::integer'
df_dc_prep = pd.read_sql_query( tSQL,con=conn )
df_dc_prep.set_index('blk_height')
df_dc_prep.info()

In [ ]:
tbase_win = 0
tdif_interval = 2016

#df_dc_prep[32000:34000]

ttime = 322   # 322  max using this setup
tbase_win = ttime*tdif_interval + 1  # skip the genesis block
tend_win  = tbase_win + tdif_interval

df_dc_prep[ tbase_win:tend_win ]
